In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import openai
from flask import Flask, request, jsonify

In [7]:
data = pd.read_csv("products.csv")
data.head(3)

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."


In [8]:
columns_to_combine = ['product', 'category', 'description']
for column in columns_to_combine:
    data[column] = data[column].fillna('')

In [9]:
data['combined_features'] = data['product'] + " " + data['category'] + " " + data['description']
data['combined_features'].head(3)

0    Garlic Oil - Vegetarian Capsule 500 mg Beauty ...
1    Water Bottle - Orange Kitchen, Garden & Pets E...
2    Brass Angle Deep - Plain, No.2 Cleaning & Hous...
Name: combined_features, dtype: object

In [10]:
data['combined_features'].isnull().sum()

np.int64(0)

In [11]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(data['combined_features'])

In [12]:
def get_recommendations(user_input):
    user_query_vector = vectorizer.transform([user_input])

    similarities = cosine_similarity(user_query_vector, tfidf_matrix).flatten()
    
    top_indices = similarities.argsort()[-5:][::-1]
    
    return data.iloc[top_indices][['product', 'sale_price']]


In [13]:
user_input = "gift for mom, New Year's decor"
recommendations = get_recommendations(user_input)
recommendations

,product,sale_price
17170,Charcoal Charmers Gift Box,1470.00
24883,Mom's Magic Cookies - Cashew & Almond,91.35
26027,Chinese Fried Rice,65.00
11009,"Brass Deepa Matki - Round, No.1",239.00
17476,"Brass Deepa Matki - Round, No.2",285.00


In [17]:
openai.api_key = 'API key'

In [15]:
import time

def generate_gift_description(product_name, recipient):
    prompt = f"Suggest a gift based on the product {product_name} for {recipient}."
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt}],
            max_tokens=50
        )
        return response['choices'][0]['message']['content'].strip()
    except openai.error.RateLimitError:
        print("Rate limit exceeded. Waiting...")
        time.sleep(10)
        return generate_gift_description(product_name, recipient)



In [ ]:
#example
gift_description = generate_gift_description( "New Year's decor", "mom")
gift_description

Rate limit exceeded. Waiting...
Rate limit exceeded. Waiting...
Rate limit exceeded. Waiting...


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/recommend', methods=['POST'])
def recommend():
    user_query = request.json.get('query')
    recommendations = get_recommendations(user_query)
    return jsonify(recommendations.to_dict(orient='records'))

@app.route('/generate_description', methods=['POST'])
def generate_description():
    product_name = request.json.get('product_name')
    recipient = request.json.get('recipient')
    description = generate_gift_description(product_name, recipient)
    return jsonify({'description': description})

if __name__ == '__main__':
    app.run(debug=True)
